In [1]:
%load_ext watermark
import pandas as pd
import numpy as np
from typing import Type, Optional, Callable
from typing import List, Dict, Union, Tuple
from myst_nb import glue

# from review_methods_tests import collect_vitals, find_missing, find_missing_loc_dates
# from review_methods_tests import make_a_summary

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

import setvariables as conf_
import reportclass as r_class

# Cumulative reports

Cumulative reports display the test statistic of a group of codes between members of a group and the cumulative value for the group. Cumulative reports are visualized with heat maps. A valid `ReportClass` object, a more detailed exmamination of the methods in the report class is availablle in [reportitng](reporting). This section refers to the production of tabular data from the observations at different geographic or administrative boundaries.

The tablular data is destined to be consumed by organizations and administrations responsible for the management or assessment of the quanity of litter in the environment. The granularity of the results are at a minimum the municipal level. The lowest recognized administrative unit.

```{note}
The GPT assistant is being trained to accept the following methods and commands as key word arguments. So that different clients have access to a standardized output for tabular data.
```

## A top level description

A short and detailed summary of the report can be created by synthesising three tables from the `ReportClass`.

```python
header = a_report.a_short_description
components = a_report.the_number_of_attributes_in_a_feature('feature_type')
comp_summary = a_report.summarize_feature_labels(feature='feature_type')
```

In [2]:
# starting data, can be MySQL or NoSQL calls
# the three methods accept Callables, as long
# as the out put is pd.DataFrame
c_l = r_class.language_maps()
surveys = r_class.collect_survey_data_for_report()
codes, beaches, land_cover, land_use, streets, river_intersect_lakes = r_class.collect_env_data_for_report()

survey_data = surveys.merge(beaches['canton'], left_on='slug', right_index=True, validate='many_to_one')

# temporal and geographic boundaries
# user defined input
boundaries = dict(canton='Bern', language='fr', start_date="2015-11-01", end_date="2021-12-31")

# the report_data method takes the boundaries and returns returns the top level of the report
# the language and two data frames from the same date range. w_df includes only the surveys
# that meet the criteria in boundaries, w_di includes all the data from the date range.
top_label, language, w_df, w_di = r_class.report_data(boundaries, survey_data.copy(), beaches, codes)

# the language map is included with the module
a_report = r_class.ReportClass(w_df,boundaries, top_label, 'fr', c_l)

# a summary of the different features and boundaries in a report
a_report.available_features

['parent_boundary', 'feature_type', 'feature_name', 'city']

In [3]:
most_common, w_mc = a_report.most_common

header = a_report.a_short_description
glue('header', header, display=False)

components = a_report.the_number_of_attributes_in_a_feature('feature_type')
glue('components', components, display=False)

comp_summary = a_report.summarize_feature_labels(feature='feature_type')
glue('comp_summary', comp_summary, display=False)

### Synthesising table data

Calling `a_report.a_short_description`, `a_report.the_number_of_attributes_in_a_feature` and `a_report.summarize_feature_labels` provides the data for a top level description and a comparison of the sample total pcs/m of different attributes or features in the data.

::::{grid}
:::{grid-item}
There were `13'759` objects identified in the period between `2015-11-01` and `2021-12-31` in the `canton` of `Bern`. In total, `196` samples were recorded, `99` on `lakes`, `96` at `rivers` and `1` at `parks`.  The lake samples were recorded from `14` `cities` and `14` for `rivers`. The `alpes` only had `one` representative. The `one` sample from the `alpes` is very close to what would be expected from a `lake` sample. The `rivers` have the `lowest` pcs/m.
:::
::::

::::{grid}
:::{grid-item}
{glue}`header`
:::
:::{grid-item}
{glue}`components`
:::

:::{grid-item}
{glue}`comp_summary`
:::
::::


```{note} Defining summary texts

Summary texts can be composed in many different ways. There is a role for LLMs at this part of the report. At the very least helping find other locations that fall within the same values. Providing context to either the geographic or adminstrative boundaries.

The person establishing the report can manage the content creation from the LLM or there can be a shared prompt for all reports.

```


### Feature names

The feature names are the labels for the various subreports that can be generated from the current set of data. The prior example is bielersee. In IQAASL the selection was `{'feature_type':'l'}` for most of the report. The selection `{'feature_type':'r'}` was the last section and `{'feature_type':'p'}` had its own chapter.

```python
a_report.features
```
The features property is a dictionary or .json object that contains the key value pairs that describe the data in the report. This includes the sample ids for each survey. 

In [4]:
# the lakes 
my_labels = a_report.feature_labels()
print(my_labels['l']['feature_name'])

['thunersee' 'bielersee' 'aare' 'brienzersee']


In [5]:
# in the same way the name of the parks and the cities in those parks can be indentified
print(my_labels['p']['feature_name'])

['alpes-bernoises']


In [6]:
# the same for rivers
print(my_labels['r']['feature_name'])

['aarenidau-buren-kanal' 'aare' 'emme' 'langeten' 'schuss' 'zulg']


## Comparing results

Once the the objects of intereste are identified (criteria) they can be compared accross the diferent feature_types and labels.

```python
t = a_cumulative_report(w_df[w_df.code.isin(most_common.index)], feature_name='feature_type', object_column='code')
translated_and_style_for_display(t, a_report.lang_maps[a_report.language], a_report.language, gradient=True)
``` 
For example the most common objects are found at different densitiies depending on the feature type.

In [7]:
t= r_class.a_cumulative_report(w_df[w_df.code.isin(most_common.index)], feature_name='feature_type', object_column='code')
r_class.translated_and_style_for_display(t, a_report.lang_maps[a_report.language], a_report.language, gradient=True)

,Cumulé,Lac,Parc,Rivière
Autres Textiles,"0,00","0,00","0,00","0,00"
Verre Brisé,"0,00","0,02","0,00","0,00"
"Fragments De Verre Ou De Céramique > 2,5 Cm","0,00","0,00","0,02","0,00"
Mégots Et Filtres À Cigarettes,"0,07","0,10","1,25","0,05"
"Emballages De Bonbons, De Snacks","0,04","0,08","0,72","0,01"
"Bâche, Feuille Plastique Industrielle","0,03","0,13","0,00","0,00"
Mousse De Plastique Pour L'Isolation Thermique,"0,00","0,00","0,00","0,00"
Emballage Plastique Non Alimentaire,"0,00","0,00","0,00","0,00"
"Couvercles En Plastique Bouteille: G21, G22, G23, G24","0,00","0,03","0,04","0,00"
"Fragments De Polystyrène Expansé: G76, G81, G82, G83","0,00","0,07","0,00","0,00"


### Alternate object groups `groupname`

If the column has other labeled values for object identification it can be used to aggregate results for each sample id. Here we consider `groupname`, there is more than one object in a group. They represent use cases.

```python
t = a_cumulative_report(w_df[w_df.code.isin(a_report.most_common.index)], feature_name='feature_type', object_column='groupname')
translated_and_style_for_display(t, a_report.lang_maps[a_report.language], a_report.language, gradient=True)
``` 
For example the different use cases are found at different densitiies depending on the feature type.

In [8]:
t = r_class.a_cumulative_report(w_df, feature_name='feature_type', object_column='groupname')
r_class.translated_and_style_for_display(t, a_report.lang_maps[a_report.language], a_report.language, gradient=True)

,Cumulé,Lac,Parc,Rivière
Agriculture,"0,06","0,14","0,00","0,02"
Nourriture Et Boissons,"0,21","0,28","0,95","0,11"
Infrastructures,"0,11","0,21","0,07","0,05"
Micro-Plastiques (< 5Mm),"0,00","0,00","0,21","0,00"
Emballage Non Alimentaire,"0,06","0,07","0,00","0,05"
Articles Personnels,"0,04","0,04","0,04","0,02"
Morceaux De Plastique,"0,09","0,32","0,12","0,00"
Loisirs,"0,04","0,10","0,06","0,00"
Tabac,"0,11","0,14","1,30","0,06"
Non Classé,"0,02","0,03","0,02","0,00"


### By Survey area or `parent_boundary`

There are two parent boundaries in the Valais, the _Alpes and Jura_ and the _Rhône_ river basin.

In [9]:
t = r_class.a_cumulative_report(w_df[w_df.code.isin(most_common.index)], feature_name='parent_boundary', object_column='code')

r_class.translated_and_style_for_display(t,a_report.lang_maps[a_report.language], a_report.language, gradient=True)

,Aare,Cumulé,Alpes Et Jura
Autres Textiles,"0,00","0,00","0,00"
Verre Brisé,"0,00","0,00","0,00"
"Fragments De Verre Ou De Céramique > 2,5 Cm","0,00","0,00","0,02"
Mégots Et Filtres À Cigarettes,"0,07","0,07","1,25"
"Emballages De Bonbons, De Snacks","0,04","0,04","0,72"
"Bâche, Feuille Plastique Industrielle","0,03","0,03","0,00"
Mousse De Plastique Pour L'Isolation Thermique,"0,00","0,00","0,00"
Emballage Plastique Non Alimentaire,"0,00","0,00","0,00"
"Couvercles En Plastique Bouteille: G21, G22, G23, G24","0,00","0,00","0,04"
"Fragments De Polystyrène Expansé: G76, G81, G82, G83","0,00","0,00","0,00"


### By `feature_name`:

There are three different features with samples: Tha Alpes, the Rhône river and Lake Geneva.

In [10]:
t = r_class.a_cumulative_report(w_df[w_df.code.isin(most_common.index)], feature_name='feature_name', object_column='code')
r_class.translated_and_style_for_display(t, a_report.lang_maps[a_report.language], a_report.language, gradient=True)

,Aare,Aarenidau-Buren-Kanal,Cumulé,Alpes-Bernoises,Bielersee,Brienzersee,Emme,Langeten,Schuss,Thunersee,Zulg
Autres Textiles,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
Verre Brisé,"0,00","0,03","0,00","0,00","0,02","0,00","0,00","0,00","0,15","0,02","0,00"
"Fragments De Verre Ou De Céramique > 2,5 Cm","0,00","0,00","0,00","0,02","0,00","0,00","0,00","0,29","0,00","0,00","0,00"
Mégots Et Filtres À Cigarettes,"0,03","0,28","0,07","1,25","0,06","0,06","0,11","0,07","0,27","0,14","0,00"
"Emballages De Bonbons, De Snacks","0,00","0,01","0,04","0,72","0,16","0,27","0,00","0,07","0,08","0,03","0,06"
"Bâche, Feuille Plastique Industrielle","0,01","0,00","0,03","0,00","0,30","0,47","0,00","0,00","0,00","0,04","0,00"
Mousse De Plastique Pour L'Isolation Thermique,"0,00","0,00","0,00","0,00","0,00","0,05","0,00","0,00","0,00","0,00","0,00"
Emballage Plastique Non Alimentaire,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,03","0,00","0,00"
"Couvercles En Plastique Bouteille: G21, G22, G23, G24","0,00","0,00","0,00","0,04","0,06","0,11","0,00","0,00","0,01","0,00","0,00"
"Fragments De Polystyrène Expansé: G76, G81, G82, G83","0,00","0,00","0,00","0,00","0,05","0,15","0,00","0,00","0,00","0,08","0,00"


### By `city`:

In [11]:
t = r_class.a_cumulative_report(w_df[w_df.code.isin(most_common.index)], feature_name='city', object_column='code')
r_class.translated_and_style_for_display(t, a_report.lang_maps[a_report.language], a_report.language, gradient=True)

,Beatenberg,Belp,Bern,Biel/Bienne,Brienz (Be),Brügg,Burgdorf,Bönigen,Erlach,Gals,Grindelwald,Kallnach,Köniz,Langenthal,Ligerz,Lüscherz,Muri Bei Bern,Nidau,Port,Rubigen,Spiez,Steffisburg,Thun,Unterseen,Utzenstorf,Vinelz,Walperswil,Cumulé
Autres Textiles,"0,00","0,00","0,00","0,00","0,00","0,00","0,04","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,04","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
Verre Brisé,"0,02","0,00","0,00","0,07","0,00","0,36","0,00","0,00","0,02","0,07","0,00","0,00","0,00","0,00","0,20","0,20","0,00","0,12","0,01","0,00","0,00","0,00","0,00","0,02","0,00","0,00","0,00","0,00"
"Fragments De Verre Ou De Céramique > 2,5 Cm","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,02","0,00","0,14","0,29","0,00","0,00","0,21","0,00","0,00","0,00","0,06","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
Mégots Et Filtres À Cigarettes,"0,55","0,00","0,02","1,15","0,00","0,28","0,07","1,19","0,44","0,09","1,25","0,00","0,59","0,07","1,00","0,04","0,54","0,00","0,63","0,06","0,08","0,00","0,23","0,55","0,11","0,04","0,00","0,07"
"Emballages De Bonbons, De Snacks","0,12","0,00","0,00","0,34","0,39","0,00","0,02","0,06","0,05","0,12","0,72","0,08","0,05","0,07","0,57","0,02","0,06","0,60","0,07","0,11","0,00","0,06","0,09","0,11","0,00","0,11","0,00","0,04"
"Bâche, Feuille Plastique Industrielle","0,03","0,02","0,02","0,18","0,67","0,00","0,22","0,15","0,00","0,03","0,00","0,32","0,00","0,00","0,40","0,05","0,00","0,40","0,00","0,00","0,00","0,00","0,13","0,13","0,00","0,79","0,00","0,03"
Mousse De Plastique Pour L'Isolation Thermique,"0,42","0,00","0,00","0,04","0,02","0,00","0,00","0,10","0,00","0,00","0,00","0,00","0,00","0,00","0,40","0,00","0,00","0,00","0,01","0,00","0,00","0,00","0,00","0,09","0,00","0,00","0,19","0,00"
Emballage Plastique Non Alimentaire,"0,04","0,00","0,00","0,09","0,06","0,00","0,11","0,00","0,02","0,00","0,00","0,01","0,00","0,00","0,14","0,00","0,00","0,24","0,04","0,00","0,00","0,00","0,00","0,03","0,00","0,00","0,00","0,00"
"Couvercles En Plastique Bouteille: G21, G22, G23, G24","0,12","0,00","0,00","0,09","0,28","0,00","0,00","0,04","0,06","0,00","0,04","0,06","0,00","0,00","0,00","0,00","0,04","0,04","0,03","0,06","0,00","0,00","0,02","0,04","0,00","0,06","0,03","0,00"
"Fragments De Polystyrène Expansé: G76, G81, G82, G83","0,18","0,00","0,00","0,18","0,22","0,00","0,00","0,07","0,02","0,00","0,00","0,14","0,00","0,00","0,00","0,00","0,00","0,04","0,00","0,00","0,05","0,00","0,16","0,11","0,00","0,03","0,00","0,00"


```{note}
The available features are column names of the survey data. They represent the different geopraphic or administrative boundaries in the selected report data. 

* `parent_boundary` is a geographic boundary such as a river basin or a category such as mountains
* `feature_type` designates whether the location is at a river, lake or park
* `feature_name` is the name of the river, lake or park
```